In [17]:
import cv2
import numpy as np
import tensorflow as tf
import imutils

In [18]:
model = tf.keras.models.load_model("../model/Line_Following_CNN_v1.h5")
video_path = "video_test.mp4"

In [25]:

   
cap = cv2.VideoCapture(video_path)
is_not_last_frame = True

while is_not_last_frame:
    is_not_last_frame, frame = cap.read()
    preprocessed_frame = []
    
    if is_not_last_frame == False:
        break

    else:
        #frame size variables
        scale_percent = 30
        width = int(frame.shape[1] * scale_percent / 100)
        height = int(frame.shape[0] * scale_percent / 100)
        dim = (width, height)
        

        #pre processing frame
        rotate = imutils.rotate(frame, 180)
        gray = cv2.cvtColor(rotate, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (32,32))
        
        #make prediction
        prediction_softmax = model.predict(resized.reshape(1, 32, 32, 1))  
        
        if np.argmax(prediction_softmax) == 0:
            text = "forward"
            start_point = ( int(width/2), height - 50)
            end_point = ( int(width/2), height - 70)
         
        elif np.argmax(prediction_softmax) == 1:
            text = "left"
            start_point = (int(width/2) + 20, height - 50)
            end_point = (int(width/2), height - 50)
            
        elif np.argmax(prediction_softmax) == 2:
            text = "right"
            start_point = (int(width/2), height - 50)
            end_point = (int(width/2) + 20, height - 50)
            
            
        #show result
        frame_copy = rotate.copy()
        frame_copy = cv2.resize(frame_copy, dim)
        cv2.putText(frame_copy, text, (50,50), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 2)
        cv2.arrowedLine(frame_copy, start_point, end_point, (255,255,255), 2, tipLength = 0.5)  
        cv2.imshow("result", frame_copy)
        
        if cv2.waitKey(1) == ord("q"):
            break
            
            
cap.release()
cv2.destroyAllWindows()